In [61]:
#This file takes the directly exported jobs csv file from IBM Quantum Experience 
#and filters them based on criteria. The filtered jobs are then used to store
#the time that job took to run. This data is used to calculate the average job time.


import pandas as pd
import random
import csv
import numpy as np
import time
import math
import itertools
import random
import progressbar
import pathlib
import datetime

#Load file
df = pd.read_csv("QMachineData/jobs.csv")

In [62]:
df

,JobId,SessionId,Status,Reason,Created,Run,Program,Name,Service,Provider,Tags,QRUsage (seconds)
0,63e64ee2812c844a9a9d5da7,NaN,CANCELLED,NaN,2023-02-10T14:04:18.089Z,2023-02-10T15:18:59.15Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
1,63e64d4b42f079c640fcd321,NaN,COMPLETED,NaN,2023-02-10T13:57:30.958Z,2023-02-10T13:57:55.172Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
2,63e64d3d5a24fc0c71f34480,NaN,COMPLETED,NaN,2023-02-10T13:57:16.563Z,2023-02-10T13:57:28.34Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
3,63e64d2b812c84066e9d5d9d,NaN,COMPLETED,NaN,2023-02-10T13:56:59.372Z,2023-02-10T13:57:13.884Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
4,63e64d16ce494c5c30b83809,NaN,COMPLETED,NaN,2023-02-10T13:56:37.575Z,2023-02-10T13:56:56.36Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6386,63bb43c9413536c8c33f7b16,NaN,COMPLETED,NaN,2023-01-08T22:29:29.715Z,2023-01-08T22:32:56.717Z,circuit-runner,NaN,ibm_oslo,ibm-q/open/main,NaN,NaN
6387,63bb43a019b723aefd04ec6f,NaN,COMPLETED,NaN,2023-01-08T22:28:48.246Z,2023-01-08T22:29:26.752Z,circuit-runner,NaN,ibm_oslo,ibm-q/open/main,NaN,NaN
6388,63bb430fccb3671d34d77033,NaN,COMPLETED,NaN,2023-01-08T22:26:23.497Z,2023-01-08T22:28:46.69Z,circuit-runner,NaN,ibm_oslo,ibm-q/open/main,NaN,NaN
6389,63b5f1264322ba29ca7db726,NaN,COMPLETED,NaN,2023-01-04T21:35:34.317Z,2023-01-04T22:07:29.513Z,circuit-runner,NaN,ibm_oslo,ibm-q/open/main,NaN,NaN


In [63]:
df = df.loc[(df['Status']== "COMPLETED") & (df['Service'] == "ibm_perth")]

In [64]:
df

,JobId,SessionId,Status,Reason,Created,Run,Program,Name,Service,Provider,Tags,QRUsage (seconds)
1,63e64d4b42f079c640fcd321,NaN,COMPLETED,NaN,2023-02-10T13:57:30.958Z,2023-02-10T13:57:55.172Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
2,63e64d3d5a24fc0c71f34480,NaN,COMPLETED,NaN,2023-02-10T13:57:16.563Z,2023-02-10T13:57:28.34Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
3,63e64d2b812c84066e9d5d9d,NaN,COMPLETED,NaN,2023-02-10T13:56:59.372Z,2023-02-10T13:57:13.884Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
4,63e64d16ce494c5c30b83809,NaN,COMPLETED,NaN,2023-02-10T13:56:37.575Z,2023-02-10T13:56:56.36Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
5,63e64d055b88706e4d1dcfef,NaN,COMPLETED,NaN,2023-02-10T13:56:20.887Z,2023-02-10T13:56:35.192Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
5960,63bcb0b0ccb36723a7d7744c,NaN,COMPLETED,NaN,2023-01-10T00:26:23.933Z,2023-01-10T00:26:41.537Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
5961,63bcb0a538d5506d966777bd,NaN,COMPLETED,NaN,2023-01-10T00:26:13.28Z,2023-01-10T00:26:21.566Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
5962,63bcb09c19b723863404f04e,NaN,COMPLETED,NaN,2023-01-10T00:26:03.557Z,2023-01-10T00:26:11.56Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN
5963,63bcb08838d5506fc76777bc,NaN,COMPLETED,NaN,2023-01-10T00:25:44.068Z,2023-01-10T00:26:01.534Z,circuit-runner,NaN,ibm_perth,ibm-q-research-2/los-gatos-hs-1/main,NaN,NaN


In [60]:
print(finalCreatedTimes[87])

[2023, 2, 10, 13, 32, 60]


In [65]:
created = df['Created'].to_list()
finalCreatedTimes = []
for timeStamp in created:
    timeStamp = timeStamp.split("T")
    date = timeStamp[0]
    date = date.split("-")
    time = timeStamp[1]
    time = time.replace("Z", "")
    time = time.split(":")
    finalCreatedTimes.append([int(date[0]),int(date[1]),int(date[2]),int(time[0]),int(time[1]), int(math.floor(float(time[2])))])
    
executed = df['Run'].to_list()
finalExecutedTimes = []
for timeStamp in executed:
    timeStamp = timeStamp.split("T")
    date = timeStamp[0]
    date = date.split("-")
    time = timeStamp[1]
    time = time.replace("Z", "")
    time = time.split(":")
    finalExecutedTimes.append([int(date[0]),int(date[1]),int(date[2]),int(time[0]),int(time[1]), int(math.floor(float(time[2])))])

In [77]:
timeDiff = []
for i in range(len(created)):
    dt1 = datetime.datetime(finalCreatedTimes[i][0],finalCreatedTimes[i][1],finalCreatedTimes[i][2],finalCreatedTimes[i][3],finalCreatedTimes[i][4], finalCreatedTimes[i][5])
    dt2 = datetime.datetime(finalExecutedTimes[i][0],finalExecutedTimes[i][1],finalExecutedTimes[i][2],finalExecutedTimes[i][3],finalExecutedTimes[i][4], finalExecutedTimes[i][5])
    temp = dt2-dt1
    timeDiff.append(temp.total_seconds())

In [94]:
sumTimeDiff = []
for i in range(int(len(timeDiff)/4)):
    sumTimeDiff.append(timeDiff[i*4] + timeDiff[i*4 + 1] + timeDiff[i*4 + 2] + timeDiff[i*4 + 3])

In [96]:
len(sumTimeDiff)

1217

In [97]:
t = 0

for i in range(len(sumTimeDiff)):
    t += sumTimeDiff[i]
print(t)
print("Time: ", t/(len(sumTimeDiff))) 

262460.0
Time:  215.66146261298275


In [110]:
import statistics
 
np.std(sumTimeDiff, ddof=1)

1451.9393429799075

In [107]:
mean = sum(sumTimeDiff) /len(sumTimeDiff)
((1451.34 - mean)/1451.34) *100

85.14052788368109

In [108]:
1451.34 * .85

1233.639

## Reconstruct alt Counts

In [120]:
import pickle
filename='QMachineData/ListData/controlValPerth'
reg =pickle.load(open(filename, 'rb'))

In [121]:
len(reg)

171